<a href="https://www.kaggle.com/code/eliyahusanti/aith-automl-fedot-catboost?scriptVersionId=187327030" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **FEDOT** - Open source AutoML framework
## *Just 5 minutes of your time, Fedot does the rest 😉*

### https://github.com/aimclub/FEDOT

**Install to kaggle**

In [ ]:
!pip install fedot fedot[extra]
!pip uninstall xarray -y

In [ ]:
import pandas as pd
from fedot.api.main import Fedot
from fedot.core.pipelines.pipeline_builder import PipelineBuilder

In [ ]:
train = pd.read_csv('/kaggle/input/ai-talent-hub-ml-1/train_dataset.csv')
test = pd.read_csv('/kaggle/input/ai-talent-hub-ml-1/test_dataset.csv')
sub = pd.read_csv("/kaggle/input/ai-talent-hub-ml-1/sample_submission.csv")

**Prepare data**

In [ ]:
test.drop(columns=["ID"], inplace=True)

**Configure AutoML parameters**

In [ ]:
auto_model = Fedot(
    problem="classification",
    metric=["precision", "accuracy", "roc_auc"],
    preset="best_quality",
    with_tuning=True,
    timeout=5,
    cv_folds=10,
    seed=42,
    n_jobs=1,
    logging_level=10,
    initial_assumption=PipelineBuilder()
    .add_node(
        "scaling",
    )
    .add_node(
        "catboost",
        params={"use_eval_set": True, "use_best_model": True, "iterations": 10000, "n_jobs": -1},
    )
    .build(),
    use_pipelines_cache=False,
    use_auto_preprocessing=True
)

In [ ]:
auto_model.fit(features=train, target="target")

In [ ]:
prediction = auto_model.predict(features=test)

**Prepare sumbission file**

In [ ]:
sub.target = prediction.ravel()
sub.to_csv("submission.csv", index=False)

**Show model information**

In [ ]:
print(auto_model.return_report().head(10))

In [ ]:
auto_model.current_pipeline.show()

**Save pipeline to load and tune later**

In [ ]:
auto_model.current_pipeline.save(path="/kaggle/working/saved_pipelines", create_subdir=True, is_datetime_in_path=True)